> <p><small><small>This Notebook is made available subject to the licence and terms set out in the <a href = "http://www.github.com/google-deepmind/ai-foundations">AI Research Foundations Github README file</a>.

# **Build Your Own Small Language Model, Lab 2: Experiment with N-gram Models**

<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_1/introduction_to_language_modeling_lab_2.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

In this lab, you will build an n-gram model. An n-gram is a contiguous sequence of *n* words. An n-gram model uses these sequences to predict the probability of the next word given a preceding sequence of n-1 words (the context). The process is broken down into steps for you to follow.

## Step 1: Define dataset and break sentences into individual words

First, define the *corpus* (text) used, which in this exercise will be [AfricaGalore](https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore.json). This dataset comprises of synthetically generated paragraphs:

In [ ]:
# Packages used.
from collections import Counter, defaultdict
import random
import pandas as pd

Run the cell below to load the dataset:

In [ ]:
africa_galore = pd.read_json('https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore.json')
train_dataset = africa_galore['description']
print(f'The dataset consists of {train_dataset.shape[0]} paragraphs.')

Now inspect the first ten paragraphs:

In [ ]:
for paragraph in train_dataset[:10]:
    # `repr` returns a printable representational string.
    print(repr(paragraph), '\n')


In order to create n-grams, sentences must be converted into individual words. This is also called  *tokenization* . The simplest way to tokenize is to break sentences into individual units based on whitespace. When you tokenize the sentence `'Bimpe didn't come home yesterday'`, you'll get a list of tokens like `['Bimpe', "didn't", 'come', 'home', 'yesterday']`:

The cell below creates a `split_text` function to split the string of text into a list of tokens.

In [ ]:
def split_text(text: str) -> list[str]:
    """Splits a string into a list of words (tokens).

    Splits text on whitespace.

    Args:
        text: The input text.

    Returns:
        A list of tokens. Returns empty list if text is empty or all whitespace.
    """
    words = text.split(' ')
    return words

# Tokenize an example text with the `split_text` function.
split_text('Here\'s how you tokenize! Awesome, isn\'t it?')

This tokenizer is extremely naive. When splitting text based on whitespace alone, you will observe that the punctuation marks are now part of the token. For example, `'tokenize!'` will be a different token from `'tokenize'`. Although this function has limitations, it is still important to explore it in more detail.

Run the cell below to tokenize the first paragraph in the dataset. The same tokenizer, which creates tokens based on whitespace, will be used:

In [ ]:
print(split_text(train_dataset[0]))

## Step 2: Create n-grams out of words




The next step is to examine the n-grams in the data. Remember, n-grams are a collection of words. For example, a unigram is one word, a bigram is two words, a trigram is three words, and so on:

Run the cell below to generate the n-grams:

In [ ]:
all_unigrams=[]
all_bigrams=[]
all_trigrams=[]

def generate_ngrams(text: str, n: int) -> list[str]:
    """Generates n-grams from a given text.

    Args:
        text: The input text string.
        n: The size of the n-grams (e.g., 2 for bigrams, 3 for trigrams).

    Returns:
        A list of n-gram strings.
    """

    words = split_text(text)
    ngrams = [' '.join(words[i:i + n]) for i in range(len(words) - n + 1)]

    return ngrams


for paragraph in train_dataset:
    all_unigrams.extend(generate_ngrams(paragraph, n=1)) # n=1 means a unigram.
    all_bigrams.extend(generate_ngrams(paragraph, n=2))  # n=2 means a bigram.
    all_trigrams.extend(generate_ngrams(paragraph, n=3)) # n=3 means a trigram.


print('First 10 Unigrams:', all_unigrams[:10])
print('First 10 Bigrams:', all_bigrams[:10])
print('First 10 Trigrams:', all_trigrams[:10])

In this cell, we will inspect which n-grams appear most frequently in the dataset.
The output of the cell below is a list with a collection of *tuples*. You can read more about tuples [in this link](https://www.w3schools.com/python/python_tuples.asp). The output of the cell is in this format: `tuple(ngram, number of occurrences)`. For example, the bigram `'is a'` appears 134 times in our dataset:

In [ ]:
bigram_counts = Counter(all_bigrams)
print('Most common Bigrams')
print(bigram_counts.most_common(10))

print('Most common Trigrams')
trigram_counts=Counter(all_trigrams)
print(trigram_counts.most_common(10))


The `generate_ngrams` function defined above can now be used to get the counts of all possible n-grams in the data. This gives the numerator of the conditional probability formula: $$ \text{Count(A B)}$$

In [ ]:
def get_ngram_counts(train_dataset: list[str],
                     n: int) -> tuple[dict[str, Counter], pd.DataFrame]:
    """Generates an n-gram count matrix and a dataframe from a training dataset.

    This function takes a list of text strings (paragraphs or sentences) as
    input, generates n-grams from each text, and creates a matrix where:

    * Columns represent n-1 word contexts.
    * Rows represent possible next words (the nth word).
    * Cells contain the count of how often a next word follows a given context.

    Args:
        train_dataset: A list of text strings representing the training data.
        n: The size of the n-grams to generate (e.g., 2 for bigrams, 3 for
            trigrams).

    Returns:
        A tuple containing:
        - A dictionary where keys are (n-1)-word contexts and values are Counter
          objects storing the counts of each next word for that context.
        - A Pandas DataFrame representing the n-gram count matrix, with contexts
          as rows, next words as columns, and counts as cell values. Missing
          values are filled with 0.
    """
    ngram_counts = defaultdict(Counter)

    for paragraph in train_dataset:
        for ngram in generate_ngrams(paragraph, n):
            words = split_text(ngram)
            context = ' '.join(words[:-1])
            next_word = words[-1]
            ngram_counts[context][next_word] += 1

    matrix = {context: dict(ngram_counts[context]) for context in ngram_counts}

    # Convert to DataFrame and fill missing values with 0.
    df = pd.DataFrame(matrix).fillna(0).astype(int)

    return matrix, df

# Example usage of the function.
sample_data = ['This is a sample sentence.',
               'Another sample sentence.',
               'Split a sentence.']
ngram_counts, df = get_ngram_counts(sample_data, 2)

print('N-gram Counts Dictionary:')
print(ngram_counts)

print('\nPandas DataFrame:')
print(df)

The `Count('sample sentence.')` is 2, since the bigram `'sample sentence.'` appears twice.
In the following step, apply this function to the training dataset and explore bigram and trigram counts.


Now, run the cell below to investigate some bigram counts.

In [ ]:
bigram_counts, bigram_df = get_ngram_counts(train_dataset, n=2)
bigram_df

In this bigram count matrix, each column represents the context (n-1) and each row represents the count of a possible next word. Because language can form an enormous number of potential word combinations, most of these combinations (5347 x 5347 as seen above) never occur (e.g., `'resonates refried'`) or occur very rarely (e.g., `'that a'`). As a result, most of the entries in this matrix are zero. This phenomenon, known as *sparsity*, indicates that, while the matrix may have thousands of cells, only a small fraction contain non-zero counts.

The following cell will explore bigrams in the dataset starting with the word `'name'`:

In [ ]:
bigram_counts['name']

This shows the following bigram counts:

`'name that'`: 2

`'name in'`: 1

`'name from'`: 1

`'name means'`: 2

It can therefore be concluded that `'name'` occurs 2+1+1+2 = 6 times in the dataset:

Run the cell below to inspect trigram counts:

In [ ]:
trigram_counts, trigram_df = get_ngram_counts(train_dataset, n=3)
trigram_df

Run the cell below to check the word that follows the bigram `'a name'`:


In [ ]:
trigram_counts['a name']

This means that the bigram `'a name'` is followed only by the word `'that'` and the trigram `'a name that'`, which appears twice in the dataset.

## Step 3: Calculate Count(A)

Recall the conditional probability formula:

$$ P(\text{B} \mid \text{A}) = \frac{\text{Count(A B)}}{\text{Count(A)}} $$

As a third step, calculate $${\text{Count(A)}}$$

Based on this formula, dividing these n-gram counts by the total count of words in context (step 3) will give the desired conditional probabilities.

In [ ]:
def build_ngram_model(train_dataset: list[str], n: int) -> dict[str, dict[str, float]]:
    """Builds an n-gram language model.

    This function takes a list of text strings (paragraphs or sentences) as
    input, generates n-grams from each text using the function get_ngram_counts
    and converts them into probabilities.  The resulting model is a dictionary,
    where keys are (n-1)-word contexts and values are dictionaries mapping
    possible next words to their conditional probabilities given the context.

    Args:
        train_dataset: A list of text strings representing the training data.
        n: The size of the n-grams (e.g., 2 for a bigram model).  This argument
            is not used directly in the calculation but is included for clarity
            and consistency.

    Returns:
        A dictionary representing the n-gram language model, where keys are
        (n-1)-word contexts and values are dictionaries mapping possible next
        words to their conditional probabilities.
    """
    ngram_model = {}

    ngram_counts, _ = get_ngram_counts(train_dataset, n)

    for context, next_words in ngram_counts.items():
        context_total_count = sum(next_words.values())
        ngram_model[context] = {
            word: count / context_total_count for word, count in next_words.items()
        }

    return ngram_model

The example used in the previous step is useful when understanding the code above.

The context in this case is `name`.

Variable context_total_count is the number of times `name` occurs, i.e. 2+1+1+2 = 6.

Variable next_words is `{'that': 2, 'in': 1, 'from': 1, 'means': 2}`

Based on the conditional probability formula, you should expect the following probabilities:

`'name that'`: 2 / 6 = 0.33

`'name in'`: 1 / 6 = 0.167

`'name from'`: 1 / 6 = 0.167

`'name means'`: 2 / 6 = 0.33

A bigram model can be created to verify these probabilities:

## Bigram model





Using the model for the context `'name'` results in another dictionary with candidate words as keys and their conditional probabilities as values:

In [ ]:
bigram_model = build_ngram_model(train_dataset, n=2)

bigram_model['name']

As you can see, the n-gram model probabilities above match the expectations.


### Understanding the output of the model

If you are unclear about what happened in the cell above, it is important to understand the datatype of the model. This n-gram model is a dictionary where:

- Keys: Represent the context (for example, a unigram `'for'`, or bigram `'looking for'`).

- Values: Represent another dictionary. Its keys are all candidates words observed in the dataset, and its values are their probabilities.

Below is an example of a dictionary within a dictionary:


In [ ]:
# Example: A dictionary of student grades.
student_grades = {
    'Annie': {
        'Math': 95,
        'Science': 88,
        'History': 92
    },
    'Teju': {
        'Math': 85,
        'Science': 90,
        'History': 80
    }
}

# Accessing the dictionary for Teju.
teju_grades = student_grades['Teju']
print('Teju\'s grades:', teju_grades)

# Extracting the subjects (keys) for Teju.
subjects = list(teju_grades.keys())
print('Subjects for Teju:', subjects)

# Extracting the grades (values) for Teju.
grades = list(teju_grades.values())
print('Grades for Teju:', grades)

# Extracting the Math grades for Teju.
math_grades = teju_grades['Math']
print('Math Grade for Teju', math_grades)

## Trigram model
This step will create a trigram model and examine some probabilities:

In [ ]:
trigram_model = build_ngram_model(train_dataset, n=3)
sample_keys = list(trigram_model.keys())[:5]
for key in sample_keys:
    print(f'{key}: {trigram_model[key]}')

The probabilities of candidate words following the bigram `'a name'` are shown below:

In [ ]:
trigram_model['a name']

This means that the trigram `'a name that'` appears once in the dataset. That means the bigram `'a name'` is always followed by the word `'that'`.

What results would searching for  `'The name'` yield?

In [ ]:
trigram_model['The name']

How about `'Their name'`?

In [ ]:
trigram_model['Their name']

This creates an error. The reason is that the bigram `'Their name'` does not exist in the dataset. Therefore, it cannot be provided as context for the trigram model.


## Coding exercise

Now that you have a model, you need code to predict the next word based on a given context. You will reuse `random.choices` from the previous lab, where it randomly selected a candidate word from a list based on specified weights (probabilities). In the previous lab, the probabilities came from your mental model of English. This time, the probabilities will come from the trigram model. As a refresher, see a sample usage of `random.choices` below.


Run the cell below multiple times to see different candidate words being picked:

In [ ]:
import random

# Define a list of items.
sample_candidate_words = ['apple', 'banana', 'cherry']

# Define corresponding weights for each fruit.
# The probability of choosing each fruit is proportional to its weight.
weights = [0.2, 0.5, 0.3]

# Sample one fruit based on the weights.
# The 'k=1' parameter indicates that we're selecting one item.
chosen_fruit = random.choices(sample_candidate_words, weights=weights, k=1)[0]

print('Chosen fruit:', chosen_fruit)


To use random choice for the context `'looking for'`, you need to write code that performs the following steps:

1. Extract candidate words for a given context from the trigram_model using the `.keys()` method.

2. Extract the corresponding probability weights using the `.values()` method.

**Step 1.** Extract candidate words for a given context from the trigram_model using the `.keys()` method

In [ ]:
context = 'looking for'
candidate_words = # Enter code here.
candidate_words

In [ ]:
# @title Run this to test your code, or get a hint.
def test_candidate_words():
    hint = """
          Hint:
          ======
          The ngram model is a dictionary where each context maps to another
          dictionary of candidate words.
          To get a list of candidate words for a specific context, put the
          context in the model as ngram_model[context] and then use the .keys()
          method.
          """

    context = 'looking for'

    try:
        if candidate_words == trigram_model[context].keys():
            print('Nice! Your answer looks correct.')
        else:
            print('\033[1m\033[91mSorry, your answer is not correct.\033[0m')
            give_hints = input('Would you like a hint? Type Yes or No.')
            if give_hints.lower() in ['yes', 'y']:
                print(hint)
    except NameError:
        print('Hint: The variable \'candidate_words\' is not defined.')
        print(f'You need to write some code after candidate_words={candidate_words}')

test_candidate_words()

**Step 2.** Extract the corresponding probability weights using the `.values()` method.

In [ ]:
context = 'looking for'
weights = # Enter code here.
weights

In [ ]:
# @title Run this to test your code, or get a hint.
def test_weights():
    hint = """
          Hint:
          ======
          This is the same as the step above except this time around we are
          fetching the values, not keys. Try using .values() instead of .keys().
          """

    try:
        if list(weights) == list(trigram_model[context].values()):
            print('Nice! Your answer looks correct.')
        else:
            print('\033[1m\033[91mSorry, your answer is not correct.\033[0m')
            give_hints = input('Would you like a hint? Type Yes or No.')
            if give_hints.lower() in ['yes', 'y']:
                print(hint)
    except:
        print('Hint: The variable \'weights\' is not defined properly')
        print(f'You need to write some code after weights={weights}')

test_weights()

Put the candidate words and their weights in random.choices, and generate a next possible word for the context `'looking for'`.

Run the cell below multiple times to see different words being picked:

In [ ]:
next_word = random.choices(list(candidate_words), weights=weights)[0]
print(context, next_word)

### Generate text

Now that you have a probability distribution over n-grams, it can be used to generate the next words given a prompt.
A prompt can be any phrase or word you want to give the model as a context or a starting point. A model can generate language by randomly selecting the next word based on their probabilities conditioned on preceding words (context).

Text generation using an n-gram model is an iterative process where each newly generated word is added to the existing context. This forms the basis for predicting the next word.

Starting with an initial prompt text, the model uses the probability distribution derived from the n-gram counts to select the most likely next word. The `random.choices` method will be used again to pick the next possible word. Once this word is generated, it is added to the context and the updated sequence is used to calculate the next probability distribution. This chain-like process continues until the next num_next_words number of words is generated, allowing the model to generate text by progressively expanding the sequence one word at a time:

In [ ]:
def generate_next_n_words(n: int, ngram_model: dict[str, dict[str, float]],
                          prompt: str, num_next_words: int) -> str:
    """Generates the next 'num_next_words' words following a given prompt using
    an n-gram language model.

    This function takes an n-gram model and uses
    it to predict the most likely next words given the provided prompt. The
    generation process continues iteratively, appending predicted words to the
    prompt until the desired number of words is generated or a context is
    encountered for which the model has no predictions.

    Args:
        n: The size of the n-grams to use (e.g., 2 for a bigram model).
        ngram_model: A dictionary representing the n-gram language model.
        prompt: The starting text prompt for generating the next words.
        num_next_words: The number of words to generate following the prompt.

    Returns:
        A string containing the original prompt followed by the generated words.
        If no valid continuation is found for a given context, the function will
        return the text generated up to that point and print a message
        indicating that no continuation could be found.
    """

    generated_words = split_text(prompt)  # Split prompt into individual words.

    for _ in range(num_next_words):
        context = generated_words[-(n - 1):]  # Get last (n-1) words as context.
        context = ' '.join(context)
        if context in ngram_model:
            # Sample next word based on probabilities.
            next_word = random.choices(
                list(ngram_model[context].keys()),
                weights=ngram_model[context].values()
            )[0]

            generated_words.append(next_word)
        else:
            print('No valid continuation found. Change the prompt or '
                  'try a smaller ngram')
            break

    return ' '.join(generated_words)

#### Finish the prompt using a bigram model

1. Run the following cell multiple times to see different results using a bigram model.
2. Try a simpler prompt, preferably one that uses phrases from the training data.
3. Try different values of n. **bold text**



In [ ]:
prompt = 'Jide was hungry so she went looking for'

n = 2 # Bigram.
num_next_words = 10 # Generate next n words.
generate_next_n_words(n=n,
                      ngram_model=bigram_model,
                      prompt=prompt,
                      num_next_words=num_next_words)

#### Finish the prompt using a trigram model

1. Run the following cell multiple times to see different results using a trigram model.
2. Try a simpler prompt, preferably one that uses phrases from the training data.
3. Try different values of n.

In [ ]:
prompt = 'Jide was hungry so she went looking for'

n = 3 # Trigram.
num_next_words = 10 # Generate next n words.
generate_next_n_words(n=n,
                      ngram_model=trigram_model,
                      prompt=prompt,
                      num_next_words=num_next_words)

The different results when running the cell multiple times are because the n-gram model picks the next word from a probability distribution randomly (also called *sampling*). More probable next words have a high probability of getting picked but are not guaranteed to be picked. This gives the output its stochasticity. Stochasticity means randomness or chance.

## What happens when you increase the *n* in n-grams?

While it intuitively seems that a larger context (higher n) would lead to better quality output by capturing more long-range dependencies in language,  it quickly runs into the problem of data sparsity, that is high dimensional array with many zero values.

When moving from bigrams (pairs of words) to trigrams (triplets of words), the number of possible combinations increases exponentially, and many of these triplets rarely, if ever, appear in the training data. This means that while bigram models can cover a significant portion of common word pairs, the majority of potential word sequences in trigram and higher-order models are underrepresented. This makes it more challenging for the model to reliably predict the next word.

Consider a simple vocabulary of five words: `['I', 'love', 'to', 'eat', 'jollof']`. For bigrams, there are at most $$5 \times 5 = 25 $$ possible combinations. In reality, however, your training data might only include common pairs like `'I love'`, `'love to'`, `'to eat'`, and `'eat jollof'`. Now, when you move to trigrams (triplets of words), there are $$ 5 \times 5 \times 5 = 125$$ possible combinations. However, only a few of these - such as `'I love to'`, `'love to eat'`, and `'to eat jollof'`, will actually appear in the data.

Even with massive datasets, many of the higher-order n-grams will never appear in the training corpus. This results in many zero counts for the probabilities. As the n-gram order increases, the number of potential combinations grows exponentially. This often leads to many combinations being rare or absent in the training data. This way, making reliable probability estimation becomes more difficult.

This phenomenon can be verified in the dataset by looking at the dimensions of the bigram matrix versus trigram matrix:

In [ ]:
print('The dimensions of our bigram matrix is', bigram_df.shape)
print('The dimensions of our trigram matrix is', trigram_df.shape)

## Reflection

This is the end of **Lab 2: Experiment with N-gram Models**.

This lab provided a hands-on exploration of n-gram models for language modeling. Here are some key takeaways and reflections:

**1. Functionality:**

- You saw how n-gram models can be used to predict the next word in a sequence based on the preceding words (context).
- N-gram models are relatively simple to implement using the conditional probabilities formula to sample the next word.

**2. Data sparsity:**

- Data sparsity is a major challenge for n-gram models, especially with higher-order n-grams (trigrams or larger).
- This sparsity arises because many possible word combinations are rare or absent in real-world text data.
- You observed this in the dataset. The dimensions of the trigram matrix are significantly larger than those of the bigram matrix, resulting in more zero values.

**3. Stochasticity and text generation:**

- While the model assigns probabilities to different next words, the actual choice is stochastic (random), resulting in different outputs for multiple runs.
- While higher probabilities increase the chances of a word being picked, less frequent words can also be generated.

**4. Considerations for text generation:**

- The size of *n* can affect the generated text quality. Larger *n* might capture longer-range dependencies but can lead to data sparsity and repetitive outputs.
- The model is unable to generate text for a prompt it has not encountered in training data.

In the next part of the course, you will review some of the limitations of n-gram models.